dependencies </n>
Env :: Forecastinit
Python=3.11.5

Libraries

In [79]:
# General 
import pandas as pd
import numpy as np

# sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# skforecast 
from skforecast.datasets import load_demo_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

Input

In [80]:
Historical_data_org = pd.read_csv('Input/Actual.csv')
TimeKey_org = pd.read_csv('Input/Time.csv')
Forecast_level = ['Version.[Version Name]', 'Channel.[Channel]', 'Account.[Account]',
       'PnL.[PnL]', 'Demand Domain.[Demand Domain]', 'Region.[Region]',
       'Location.[Location]', 'Time.[Planning Month]', 'Item.[Item]']
Time_column = ['Time.[Planning Month]']
Historical_data_column = ['Actual']
#Date should be in format of time key column, which is available in TimeKey file 
Time_key_column_name = "Time.[PlanningMonthKey]"
Time_key_date_format = "%m/%d/%Y %I:%M:%S %p"
Historic_start_date = "M09-20" 
Historic_end_date = "M12-22"
Forecast_start_date = "M01-23"
Forecast_end_date = "M03-23"
drivers = ['driver_Month']


Custom Functions

In [81]:
def split(data,start_date_key,end_date_key,X_cols,y_col):
    # Data filter based on start and end date, then split in X and y based on provided columns 
    data = data[(data['TimeKey']>=start_date_key)&(data['TimeKey']<=end_date_key)]
    X = data[['key','TimeKey']+X_cols]
    y = data[[y_col]]
    return X, y

Processing Input

In [82]:
Forecast_level_minus_time = Forecast_level.copy()
Forecast_level_minus_time.remove(Time_column[0])
Forecast_level_minus_time

['Version.[Version Name]',
 'Channel.[Channel]',
 'Account.[Account]',
 'PnL.[PnL]',
 'Demand Domain.[Demand Domain]',
 'Region.[Region]',
 'Location.[Location]',
 'Item.[Item]']

Copy Input 

In [83]:
Historical_data = Historical_data_org.copy(deep=True)
TimeKey = TimeKey_org.copy(deep=True)
TimeKey['TimeKey'] = pd.to_datetime(TimeKey[Time_key_column_name], format=Time_key_date_format)

In [84]:
# calculating Historic_end_date_key
Historic_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_start_date]['TimeKey'].values[0])
Historic_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_end_date]['TimeKey'].values[0])
Forecast_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_start_date]['TimeKey'].values[0])
Forecast_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_end_date]['TimeKey'].values[0])

Processing Copy Input

In [85]:
Historical_data[Historical_data_column] = Historical_data[Historical_data_column].astype(float)


Creating Key in Historical Data

In [86]:
# creating keys 
Historical_data['key'] = Historical_data[Forecast_level_minus_time].astype(str).agg("__ MDJoinner__".join, axis=1)
# dropping columns, which is already present in keys 
Historical_data.drop(Forecast_level_minus_time,axis=1,inplace=True) 
Historical_data.head(2)

,Time.[Planning Month],Actual,key
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...


Custom Filter

In [87]:
Historical_data = Historical_data[Historical_data['key']=='CurrentWorkingView__ MDJoinner__B2B__ MDJoinner__AMO__ MDJoinner__DP__ MDJoinner__DP__ MDJoinner__ShipTo1__ MDJoinner__DP__ MDJoinner__Loctite 248 19g Stick']
# Historical_data['key'].values

Merging Time Key in Historical Data

In [88]:
Historical_data = pd.merge(Historical_data,TimeKey[Time_column+["TimeKey"]],on=Time_column,how='left')
Historical_data.head(2)

,Time.[Planning Month],Actual,key,TimeKey
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02


In [89]:
# Creating drivers 
Historical_data['driver_Month'] = Historical_data['TimeKey'].dt.month
Historical_data.head()

,Time.[Planning Month],Actual,key,TimeKey,driver_Month
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05,7
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02,8
2,M09-20,462.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-30,8
3,M10-20,174.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-10-04,10
4,M11-20,179.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-01,11


Train and Test Data

In [91]:
X_train, y_train = split(Historical_data, Historic_start_date_key,Historic_end_date_key,drivers,Historical_data_column[0])
X_test,y_test = split(Historical_data, Forecast_start_date_key, Forecast_end_date_key, drivers, Historical_data_column[0])

,key,TimeKey,driver_Month
30,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2023-01-01,1
31,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2023-01-29,1
32,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2023-02-26,2


In [53]:
from sklearn.preprocessing import FunctionTransformer
transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = [1]
             )

forecaster.fit(y=y_train[Historical_data_column[0]])
forecaster

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


ForecasterAutoreg 
Regressor: RandomForestRegressor(random_state=123) 
Lags: [1] 
Transformer for y: None 
Transformer for exog: None 
Window size: 1 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: [0, 27] 
Training index type: RangeIndex 
Training index frequency: 1 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-01-23 19:12:27 
Last fit date: 2024-01-23 19:12:27 
Skforecast version: 0.11.0 
Python version: 3.11.5 
Forecaster id: None 

In [54]:
# Predict intervals
# predictions = 
# forecaster.predict_interval(steps=1)
forecaster.predict_interval(steps=11)
# predictions.head(1)

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


,pred,lower_bound,upper_bound
1,251.0400,182.140000,417.175500
2,230.6350,154.260500,541.190500
3,177.3795,143.727500,569.124250
4,229.8500,144.845500,590.690000
5,153.8595,160.200775,598.240000
6,191.9200,142.487000,612.869350
7,252.3400,143.407500,595.214750
8,230.6350,152.443225,583.800500
9,177.3795,160.534750,570.258750
10,229.8500,144.255475,671.191275


In [55]:
from sklearn.preprocessing import FunctionTransformer
transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = [1,4]
             )

forecaster.fit(y=y_train[Historical_data_column[0]])
forecaster

forecaster.predict_interval(steps=11)

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


,pred,lower_bound,upper_bound
4,216.75,92.0300,308.0500
5,285.24,214.5900,606.8000
6,531.74,293.3930,729.8070
7,431.16,281.4105,534.0000
8,400.30,149.9600,596.5975
9,195.77,106.1250,393.4685
10,210.03,92.6475,324.5280
11,184.44,95.1520,311.7225
12,216.67,94.9985,370.6460
13,246.12,125.2070,513.7630


In [62]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd

# Modelling and Forecasting
# ==============================================================================
# from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble  import StackingRegressor
from sklearn.model_selection  import KFold

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.datasets import fetch_dataset

# Warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [67]:
# Grid search of hyperparameters and lags
# ==============================================================================
param_grid = {
}

# Lags used as predictors
lags_grid = [2]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = y_train[Historical_data_column[0]],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 3,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(y_train[Historical_data_column[0]])-3,
                   fixed_train_size   = False,
                   return_best        = True,
                   n_jobs             = 'auto',
                   verbose            = True
               )

results_grid.head()

Number of models compared: 1.


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

Information of backtesting process
----------------------------------
Number of observations used for initial training: 25
Number of observations used for backtesting: 3
    Number of folds: 1
    Number of steps per fold: 3
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2 -- 26  (n=25)
    Validation: 27 -- 29  (n=3)

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2] 
  Parameters: {}
  Backtesting metric: 30221.109166666665



,lags,params,mean_squared_error
0,"[1, 2]",{},30221.109167
